<a href="https://colab.research.google.com/github/atulgupta01/EVA_Group_Assignment/blob/master/Assignment19/Copy_of_dcGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import cv2
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [0]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 512)))
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        #model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128, save_interval=50,start_epoch_no=0):

        

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        #X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch+start_epoch_no, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if ((epoch+start_epoch_no) % save_interval) == 0:
                self.save_imgs(epoch+start_epoch_no)
                self.save_model()

    def save_model(self):
          self.generator.save_weights("/content/drive/My Drive/EVA_Assignment_19/modelGAN_generatorCopy.h5")                
          self.discriminator.save_weights("/content/drive/My Drive/EVA_Assignment_19/modelGAN_discriminatorCopy.h5")                


    def load_model(self):
          self.generator.load_weights("/content/drive/My Drive/EVA_Assignment_19/modelGAN_generatorCopy.h5")                
          self.discriminator.load_weights("/content/drive/My Drive/EVA_Assignment_19/modelGAN_discriminatorCopy.h5")  

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,:], )
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/drive/My Drive/Session19images/guns_%d.png" % (epoch))
        plt.close()

In [0]:
import glob
import os

DATA_PATH = '/content/drive/My Drive/EVA_Assignment_19'
# make a list of frames in the aligned frames folder
imgList = list(glob.iglob(os.path.join(DATA_PATH, '*.*')))



In [0]:
X_train=[]
img1 = cv2.imread('/content/drive/My Drive/EVA_Assignment_19/91.jpg')
img1 = cv2.resize(img1, (64,64), interpolation = cv2.INTER_AREA)
X_train = np.expand_dims(img1,axis=0)
for imgName in imgList:
  print('Reading {}'.format(imgName))
  img = cv2.imread(imgName)
  rw,cl,ch = img.shape
  print('size->({},{})'.format(rw,cl))
  img = cv2.resize(img, (64,64), interpolation = cv2.INTER_AREA)
  img = np.expand_dims(img,axis=0)
  X_train=np.append(X_train,img,axis=0)
  

  

Reading /content/drive/My Drive/EVA_Assignment_19/83.jpg
size->(366,600)
Reading /content/drive/My Drive/EVA_Assignment_19/86.jpg
size->(1000,1000)
Reading /content/drive/My Drive/EVA_Assignment_19/87.jpg
size->(225,225)
Reading /content/drive/My Drive/EVA_Assignment_19/84.jpg
size->(194,259)
Reading /content/drive/My Drive/EVA_Assignment_19/85.jpg
size->(225,225)
Reading /content/drive/My Drive/EVA_Assignment_19/91.jpg
size->(224,224)
Reading /content/drive/My Drive/EVA_Assignment_19/88.jpg
size->(225,225)
Reading /content/drive/My Drive/EVA_Assignment_19/92.jpg
size->(225,225)
Reading /content/drive/My Drive/EVA_Assignment_19/89.jpg
size->(458,458)
Reading /content/drive/My Drive/EVA_Assignment_19/90.jpg
size->(375,375)
Reading /content/drive/My Drive/EVA_Assignment_19/93.jpg
size->(496,661)
Reading /content/drive/My Drive/EVA_Assignment_19/97.jpg
size->(1200,1200)
Reading /content/drive/My Drive/EVA_Assignment_19/94.jpg
size->(225,225)
Reading /content/drive/My Drive/EVA_Assignment_

AttributeError: ignored

In [0]:
dcgan = DCGAN()

dcgan.train(X_train, epochs=4000, batch_size=32, save_interval=50)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 64)        0    

KeyboardInterrupt: ignored

In [0]:
dcgan = DCGAN()
dcgan.load_model()

dcgan.train(X_train, epochs=4000, batch_size=32, save_interval=50,start_epoch_no=1)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_10 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 16, 16, 64)       

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1020 [D loss: 0.884710, acc.: 62.50%] [G loss: 2.239958]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1021 [D loss: 0.111507, acc.: 96.88%] [G loss: 4.840251]
1022 [D loss: 0.780687, acc.: 57.81%] [G loss: 2.519314]
1023 [D loss: 0.656842, acc.: 75.00%] [G loss: 3.776372]
1024 [D loss: 0.198959, acc.: 93.75%] [G loss: 3.485632]
1025 [D loss: 0.568985, acc.: 70.31%] [G loss: 3.143655]
1026 [D loss: 0.173976, acc.: 98.44%] [G loss: 4.074000]
1027 [D loss: 0.325928, acc.: 89.06%] [G loss: 3.588336]
1028 [D loss: 0.420398, acc.: 79.69%] [G loss: 2.809765]
1029 [D loss: 0.357959, acc.: 85.94%] [G loss: 3.538410]
1030 [D loss: 0.155805, acc.: 98.44%] [G loss: 2.949097]
1031 [D loss: 0.525502, acc.: 78.12%] [G loss: 4.009979]
1032 [D loss: 0.263236, acc.: 89.06%] [G loss: 4.205490]
1033 [D loss: 0.527381, acc.: 75.00%] [G loss: 3.551036]
1034 [D loss: 0.328756, acc.: 87.50%] [G loss: 3.767285]
1035 [D loss: 0.431808, acc.: 75.00%] [G loss: 2.293175]
1036 [D loss: 0.357438, acc.: 82.81%] [G loss: 4.456250]
1037 [D loss: 0.590288, acc.: 76.56%] [G loss: 3.216221]
1038 [D loss: 0.446631, acc.: 7

KeyboardInterrupt: ignored

In [0]:
dcgan = DCGAN()
dcgan.load_model()

dcgan.train(X_train, epochs=4000, batch_size=32, save_interval=50,start_epoch_no=1821)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_19 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 16, 16, 64)       

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1821 [D loss: 0.116415, acc.: 98.44%] [G loss: 4.777758]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1822 [D loss: 0.016272, acc.: 100.00%] [G loss: 6.340481]
1823 [D loss: 0.032756, acc.: 100.00%] [G loss: 8.268696]
1824 [D loss: 0.099414, acc.: 98.44%] [G loss: 7.938467]
1825 [D loss: 0.006904, acc.: 100.00%] [G loss: 9.570461]
1826 [D loss: 0.019819, acc.: 100.00%] [G loss: 5.924885]
1827 [D loss: 0.030115, acc.: 100.00%] [G loss: 6.697103]
1828 [D loss: 0.025521, acc.: 100.00%] [G loss: 5.422039]
1829 [D loss: 0.046987, acc.: 100.00%] [G loss: 6.880138]
1830 [D loss: 0.008784, acc.: 100.00%] [G loss: 6.579162]
1831 [D loss: 0.045588, acc.: 100.00%] [G loss: 6.929455]
1832 [D loss: 0.164393, acc.: 96.88%] [G loss: 8.725731]
1833 [D loss: 0.047802, acc.: 98.44%] [G loss: 5.370621]
1834 [D loss: 0.020450, acc.: 100.00%] [G loss: 6.649355]
1835 [D loss: 0.016555, acc.: 100.00%] [G loss: 7.024532]
1836 [D loss: 0.032759, acc.: 98.44%] [G loss: 6.623753]
1837 [D loss: 0.033202, acc.: 100.00%] [G loss: 7.434024]
1838 [D loss: 0.010775, acc.: 100.00%] [G loss: 6.296946]
1839 [D loss: 0.19

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


2551 [D loss: 0.020001, acc.: 100.00%] [G loss: 5.998769]
2552 [D loss: 0.004118, acc.: 100.00%] [G loss: 6.427567]
2553 [D loss: 0.053704, acc.: 100.00%] [G loss: 5.134659]
2554 [D loss: 0.036714, acc.: 100.00%] [G loss: 4.794304]
2555 [D loss: 0.080617, acc.: 98.44%] [G loss: 7.699808]
2556 [D loss: 0.040425, acc.: 100.00%] [G loss: 8.240269]
2557 [D loss: 0.017532, acc.: 100.00%] [G loss: 5.417544]
2558 [D loss: 0.014942, acc.: 100.00%] [G loss: 6.165301]
2559 [D loss: 0.133164, acc.: 95.31%] [G loss: 5.514303]
2560 [D loss: 0.029868, acc.: 100.00%] [G loss: 6.147516]
2561 [D loss: 0.016840, acc.: 100.00%] [G loss: 6.537527]
2562 [D loss: 0.158094, acc.: 96.88%] [G loss: 4.717266]
2563 [D loss: 0.007538, acc.: 100.00%] [G loss: 7.916245]
2564 [D loss: 0.002524, acc.: 100.00%] [G loss: 5.352021]
2565 [D loss: 0.024591, acc.: 100.00%] [G loss: 4.253024]
2566 [D loss: 0.053250, acc.: 100.00%] [G loss: 4.784410]
2567 [D loss: 0.050125, acc.: 100.00%] [G loss: 8.424309]
2568 [D loss: 0.0

KeyboardInterrupt: ignored